# Simple RNN Tensorflow

In [46]:
# this cell is for imports
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import pandas as pd
import collections
import os
import time
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os, sys
import string
sys.path.insert(0, "/content/gdrive/My Drive/Project-Lion/")
root_folder = "/content/gdrive/My Drive/Project-Lion/"
# getting jokes from kaggle dataset
df = pd.read_csv(root_folder + 'data/kaggle/shortjokes.csv')
df = df.loc[df["Joke"] != ""]

In [0]:
def clean_data(lst, in_place=True, keep_punc=""):
    if in_place:
        new_lst = lst
    else:
        new_lst = lst.copy()
        
    for i in range(len(new_lst)):
        joke = lst[i]
        joke = joke.replace("\r", " ").replace("\n", " ").replace("/", " or ")
        joke = joke.translate(str.maketrans("", "", string.punctuation.replace(keep_punc, ""))) # Remove punctuation
        joke = joke.lower() # Lowercase
        joke = ''.join(char for char in joke if not char.isdigit()) # Remove numbers
        joke = joke.strip() # Remove leading and ending whitespace
        new_lst[i] = joke
    return new_lst

In [5]:
data = clean_data(df["Joke"].tolist(), keep_punc="?")
qa_all = [[s.strip() for s in list(filter(None, joke.split("?")))] for joke in data if len(list(filter(None, joke.split("?")))) == 2]
print("Size of entire dataset:", len(qa_all))
print(qa_all[0])
min_qlen, max_qlen = 3, 15
min_alen, max_alen = 3, 15
qa = [(q, a) for q, a in qa_all if len(q.split()) >= min_qlen and len(q.split()) <= max_qlen and len(a.split()) >= min_alen and len(a.split()) <= max_alen]
print("Size of dataset:", len(qa))
print(qa[0])

Size of entire dataset: 93782
['why cant barbie get pregnant', 'because ken comes in a different box heyooooooo']
Size of dataset: 59999
('why cant barbie get pregnant', 'because ken comes in a different box heyooooooo')


In [0]:
questions = [q for q, a in qa]

In [9]:
# the rest of this notebook follows code from https://www.tensorflow.org/tutorials/sequences/text_generation
# build a set of vocab (characters)
joke_lst = df["Joke"].tolist()
text = joke_lst[0]
# use the first 1000 jokes to play with model
for joke in joke_lst[1:20000]:
    text += '\n\n'
    text += joke
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

97 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\x08':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}


In [12]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'[me narrating' ---- characters mapped to int ---- > [61 79 71  2 80 67 84 84 67 86 75 80 73]


In [13]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.
[
m
e
 
n


In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTell'
'ing my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vam'
"pires... men.\n\nI've been going through a really rough period at work this week It's my own fault for "
'swapping my tampax for sand paper.\n\nIf I could have dinner with anyone, dead or alive... ...I would c'
"hoose alive. -B.J. Novak-\n\nTwo guys walk into a bar. The third guy ducks.\n\nWhy can't Barbie get pregn"


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTel'
Target data: 'me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTell'


In [17]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 61 ('[')
  expected output: 79 ('m')
Step    1
  input: 79 ('m')
  expected output: 71 ('e')
Step    2
  input: 71 ('e')
  expected output: 2 (' ')
Step    3
  input: 2 (' ')
  expected output: 80 ('n')
Step    4
  input: 80 ('n')
  expected output: 67 ('a')


In [18]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [0]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           24832     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [26]:
sampled_indices

array([67,  0, 16, 90, 52, 24, 76, 77,  9, 67, 35,  9, 67, 77, 51, 23, 18,
        2, 89,  4, 68,  4, 20, 87, 61, 61, 12, 86, 40, 53, 84, 87, 11, 41,
        5, 52, 57, 92, 11, 17, 43, 12, 21, 57, 28, 15, 16, 76, 14, 42,  3,
        6, 55, 85, 93, 22, 94, 90, 93, 53, 83, 59, 68, 82, 19, 23, 26,  9,
       65, 26, 32, 69, 27,  2, 51, 81, 78,  4,  4, 43, 19, 94,  7,  5, 10,
       17, 27, 73, 37, 50, 62, 93, 78, 68, 83, 32, 96, 84, 13, 20])

In [27]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 't\n\nWhat do you call a fly with no wings? A walk... budumdum tssssss\n\nUsing a macbook is like making '

Next Char Predictions: 
 'a\x08.xR6jk\'aA\'akQ50 w"b"2u[[*tFSru)G#RWz)/I*3W:-.j,H!$Us{4|x{SqYbp158\'_8>c9 Qol""I1|%#(/9gCP\\{lbq>~r+2'


In [28]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5756917


In [0]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [32]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])


Epoch 1/10
297/298 [============================>.] - ETA: 0s - loss: 2.5365WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
298/298 [==============================] - 27s 91ms/step - loss: 2.5347
Epoch 2/10
298/298 [==============================] - 24s 82ms/step - loss: 1.8499
Epoch 3/10
298/298 [==============================] - 24s 81ms/step - loss: 1.6225
Epoch 4/10
298/298 [==============================] - 24s 81ms/step - loss: 1.5079
Epoch 5/10
298/298 [==============================] - 24s 82ms/step - loss: 1.4329
Epoch 6/10
298/298 [==============================] - 24s 81ms/step - loss: 1.3785
Epoch 7/10
298/298 [==============================] - 24s 81

In [33]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            24832     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 97)             99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [0]:
from ast import literal_eval
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
        
    question = start_string.split(" ")
    text_generated = text_generated[len(question):]
    sentence = ''.join(text_generated)
    
    sentence = sentence.replace('\n','')

    return sentence

In [0]:
questions_500 = questions[:500]
#print(questions_500)

In [0]:
with open(root_folder + "LSTM_output_jokes.txt", "w") as text_file:
  for question in questions_500:
    print(question, file = text_file)
    print(generate_text(model, start_string=question), file = text_file)
    print("", file = text_file)

In [84]:
print(generate_text(model, start_string="Why did the chicken "))

 the road? Because he couldn't find their dramatic out the world he asked him what do you call 
